In [21]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = list(glob.glob('../features/*.pkl'))
for l in sorted(fl):
    print("'{}'".format(l))

'../features/cnn_1_aug1_feat.pkl'
'../features/cnn_1_aug_1_fold8_feat.pkl'
'../features/cnn_1_aug_rescale_preprocess_feat.pkl'
'../features/cnn_1_aug_skimage_denoise_feat.pkl'
'../features/cnn_1_aug_skimage_preprocess_feat.pkl'
'../features/cnn_1_feat.pkl'
'../features/cnn_2_aug1_feat.pkl'
'../features/cnn_2_aug_1_fold8_feat.pkl'
'../features/cnn_2_aug_denoise_preprocess_feat.pkl'
'../features/cnn_2_aug_rescale_preprocess_feat.pkl'
'../features/cnn_2_aug_skimage_preprocess_feat.pkl'
'../features/cnn_2_feat.pkl'
'../features/cnn_3_aug1_feat.pkl'
'../features/cnn_3_aug_1_fold8_feat.pkl'
'../features/cnn_3_aug_denoise_preprocess_feat.pkl'
'../features/cnn_3_aug_rescale_preprocess_feat.pkl'
'../features/cnn_3_aug_skimage_preprocess_feat.pkl'
'../features/cnn_4_aug1_feat.pkl'
'../features/cnn_4_aug1_feat_add_early.pkl'
'../features/cnn_4_aug_1_fold8_feat.pkl'
'../features/cnn_4_aug_denoise_preprocess_feat.pkl'
'../features/cnn_4_aug_rescale_preprocess_feat.pkl'
'../features/cnn_4_aug_skimag

In [22]:
def load_x_y():
    train_x,test_x=[],[]
    for f in fl:
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

(1604, 61) (8424, 61) (1604,)


In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
def cv_test(k_cnt=3,rnd=42,save_flag=False):
    kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd*2)
    w_test_pred = None
    r_val_loss = 0
    test_pred = None
    val_loss = 0
    val_loss_list = []
    for train_index, test_index in kf.split(train_x,train_y):
        X_train, X_test = train_x[train_index], train_x[test_index]
        y_train, y_test = train_y[train_index], train_y[test_index]
        params = {
                'colsample_bytree': 0.75,
                'colsample_bylevel':0.95,
                'gamma':0.2,
                'subsample': 0.9,
                'eta': 0.07,
                'max_depth': 3,
                'eval_metric':'logloss',
                'objective':'binary:logistic',
                'scale_pos_weight': 0.85
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(test_x)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 500, watchlist, 
                        early_stopping_rounds=20,
                        verbose_eval=1000)
        
        # cal val loss
        curr_val_loss = log_loss(y_test,m.predict(d_valid))
        val_loss += curr_val_loss/k_cnt
        val_loss_list.append(curr_val_loss)
        curr_r_val_loss = 1.0 / curr_val_loss
        r_val_loss += curr_r_val_loss
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            w_test_pred = m.predict(d_test)*curr_r_val_loss
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            w_test_pred = w_test_pred + curr_pred*curr_r_val_loss

    # avg
    test_pred = test_pred / k_cnt
    w_test_pred = w_test_pred / r_val_loss
    
    # train log loss
    print('local average valid loss',val_loss,'val loss std',np.std(val_loss_list))
    if save_flag:
        test_df=pd.read_json('../input/test.json')
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=test_pred
        print(submission.head())
        submission.to_csv('../results/all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)
        
        submission = pd.DataFrame()
        submission['id']=test_df['id']
        submission['is_iceberg']=w_test_pred
        print(submission.head())
        submission.to_csv('../results/weighted_all_xgb_sub_fold_{}_rnd_{}.csv'.format(k_cnt,rnd), index=False)

    
print('def done')

def done


In [24]:
cv_test(3,42,True)

[0]	train-logloss:0.637035	valid-logloss:0.640824
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[126]	train-logloss:0.047605	valid-logloss:0.151843

[0]	train-logloss:0.636924	valid-logloss:0.640642
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[116]	train-logloss:0.052841	valid-logloss:0.15823

[0]	train-logloss:0.6373	valid-logloss:0.638559
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[69]	train-logloss:0.080784	valid-logloss:0.159995

local average valid loss 0.158173335467 val loss std 0.00305382922753
         id  is_iceberg
0  5941774d    0.014298
1  4023181e    0.956390
2  b20200e4    0.04686

In [25]:
cv_test(5,42,True)

[0]	train-logloss:0.637039	valid-logloss:0.641443
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[95]	train-logloss:0.067321	valid-logloss:0.173268

[0]	train-logloss:0.637805	valid-logloss:0.638863
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[142]	train-logloss:0.048459	valid-logloss:0.129403

[0]	train-logloss:0.636644	valid-logloss:0.63871
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[75]	train-logloss:0.079222	valid-logloss:0.174255

[0]	train-logloss:0.638058	valid-logloss:0.641276
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-

In [26]:
cv_test(10,42,True)

[0]	train-logloss:0.637099	valid-logloss:0.642605
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[66]	train-logloss:0.091165	valid-logloss:0.194401

[0]	train-logloss:0.636875	valid-logloss:0.640842
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[96]	train-logloss:0.072941	valid-logloss:0.147053

[0]	train-logloss:0.637579	valid-logloss:0.638839
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[76]	train-logloss:0.085773	valid-logloss:0.140621

[0]	train-logloss:0.637021	valid-logloss:0.638768
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-

In [27]:
cv_test(3,233,True)

[0]	train-logloss:0.638126	valid-logloss:0.638934
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[101]	train-logloss:0.066745	valid-logloss:0.136599

[0]	train-logloss:0.637751	valid-logloss:0.639852
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[144]	train-logloss:0.045656	valid-logloss:0.137707

[0]	train-logloss:0.635512	valid-logloss:0.640866
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[71]	train-logloss:0.068168	valid-logloss:0.195558

local average valid loss 0.159071232849 val loss std 0.0293440018987
         id  is_iceberg
0  5941774d    0.011449
1  4023181e    0.969048
2  b20200e4    0.054

In [28]:
cv_test(5,233,True)

[0]	train-logloss:0.637712	valid-logloss:0.640377
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[81]	train-logloss:0.079673	valid-logloss:0.159986

[0]	train-logloss:0.637382	valid-logloss:0.635612
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[123]	train-logloss:0.061506	valid-logloss:0.100947

[0]	train-logloss:0.637627	valid-logloss:0.639512
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[126]	train-logloss:0.056221	valid-logloss:0.135502

[0]	train-logloss:0.637362	valid-logloss:0.643579
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until vali

In [29]:
cv_test(10,233,True)

[0]	train-logloss:0.637694	valid-logloss:0.639075
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[104]	train-logloss:0.067824	valid-logloss:0.136015

[0]	train-logloss:0.636316	valid-logloss:0.638591
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[132]	train-logloss:0.05428	valid-logloss:0.16425

[0]	train-logloss:0.63794	valid-logloss:0.637022
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
Stopping. Best iteration:
[145]	train-logloss:0.049658	valid-logloss:0.11995

[0]	train-logloss:0.637318	valid-logloss:0.637194
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-l